# 05 — LSTM on ARIMA Residuals
Train an LSTM to forecast residual dynamics, then combine:
forecast(corr) = forecast_arima + forecast_lstm(residual).

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models

resid_df = pd.read_parquet("../data/processed/arima_residuals_sample.parquet")
resid_df.shape

In [ ]:
# Build supervised sequences
def make_sequences(x, lookback=20):
    # x: 1D array
    X, y = [], []
    for i in range(lookback, len(x)):
        X.append(x[i-lookback:i])
        y.append(x[i])
    return np.array(X), np.array(y)

LOOKBACK = 20

# Train LSTM on one series first (keep it fast & clear)
series = resid_df.columns[0]
x = resid_df[series].values.astype("float32")

X, y = make_sequences(x, lookback=LOOKBACK)
X = X[..., None]  # (n, lookback, 1)

# time split
n = len(X)
train_end = int(0.7*n)
val_end = int(0.85*n)

X_tr, y_tr = X[:train_end], y[:train_end]
X_va, y_va = X[train_end:val_end], y[train_end:val_end]
X_te, y_te = X[val_end:], y[val_end:]
X_tr.shape, X_va.shape, X_te.shape

In [ ]:
# LSTM model
def build_lstm_model(lookback=20, units=10, dropout=0.2):
    model = models.Sequential([
        layers.Input(shape=(lookback, 1)),
        layers.LSTM(units),
        layers.Dropout(dropout),
        layers.Dense(1)
    ])
    model.compile(optimizer=tf.keras.optimizers.Adam(1e-3), loss="mse")
    return model

model = build_lstm_model(lookback=LOOKBACK, units=10, dropout=0.2)

cb = [
    tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)
]

hist = model.fit(
    X_tr, y_tr,
    validation_data=(X_va, y_va),
    epochs=50,
    batch_size=64,
    callbacks=cb,
    verbose=1
)

In [ ]:
import os
os.makedirs("../models", exist_ok=True)
model.save(f"../models/lstm_residual_{series}.keras")